In [22]:
import pandas as pd
import numpy as np

from haversine import haversine

import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv("cor_df.csv")

In [ ]:
remove_idx = []
for i in range(0, 825, 4):
    temp = []
    a, b = i, i + 3
    lst = [df.iloc[a, -1], df.iloc[a + 1, -1], df.iloc[b - 1, -1], df.iloc[b, -1]]
    for j in range(a, b + 1):
        if "없음" in df.iloc[j, -1]:
            temp = [a, a + 1, a + 2, a + 3]
    remove_idx += temp
remove_idx = set(remove_idx)

df.drop(list(remove_idx), axis = 0, inplace = True)
df.reset_index(inplace = True, drop = True)

In [2]:
remove_idx = []
for i in range(0, 825, 4):
    temp = []
    a, b = i, i + 3
    for j in range(a, b + 1):
        if "없음" in df.iloc[j, -1]:
            temp = [a, a + 1, a + 2, a + 3]
    remove_idx += temp
remove_idx = set(remove_idx)

df.drop(list(remove_idx), axis = 0, inplace = True)
df.reset_index(inplace = True, drop = True)

In [17]:
len(remove_idx)

264

In [3]:
df["X"] = 0
df["y"] = 0
for i in range(len(df)):
    X, y = df.iloc[i, -3][3:13], df.iloc[i, -3][-10:]
    df.iloc[i, -2] = X; df.iloc[i, -1] = y
    
df.drop(["cor"], axis = 1, inplace = True)
df.drop(["주소"], axis = 1, inplace = True)
df["호선"] = df["호선"].astype("int")

In [4]:
for i in range(len(df)):
    if df.loc[i, "역"][-1] == "역":
        df.loc[i, "역"] = df.loc[i, "역"][:-1]

In [5]:
new_dict = dict(df["역"].value_counts())
dup_stat = []
for k, v in new_dict.items():
    if v == 8: dup_stat += (list(df[df["역"] == k].index[:4]))
df.drop(dup_stat, axis = 0, inplace = True)
df.reset_index(inplace = True, drop = True)

In [6]:
df.sort_values(["건물", "역"], inplace = True)
D, S, O, E = df[df["건물"] == "다이소"], df[df["건물"] == "스타벅스"], df[df["건물"] == "올리브영"], df[df["건물"] == "이디야"]
for data, idx in zip([D, S, O, E], [1, 2, 3, 4]):
    data.reset_index(inplace = True, drop = True)
    if idx == 1: continue
    else:
        sub_data = data.iloc[:, -2:]
        D = pd.concat([D, sub_data], axis = 1)

In [7]:
D.drop(["건물"], axis = 1, inplace = True)
D.columns = ["Num", "Station", "D_longitude", "D_latitude", "S_longitude", "S_latitude", "O_longitude", "O_latitude", "E_longitude", "E_latitude"]
D = D[["E_latitude", "E_longitude", "D_latitude", "D_longitude", "O_latitude", "O_longitude", "S_latitude", "S_longitude", "Station", "Num"]]
df = D.copy()

In [8]:
for col in df.columns:
    if df[col].dtype == "object" and col != "Station":
        df[col] = df[col].astype('float')

In [9]:
df["ED"], df["DO"], df["OE"] = 0, 0, 0

for i in range(len(df)):
    
    e = (df.iloc[i, 0], df.iloc[i, 1])
    d = (df.iloc[i, 2], df.iloc[i, 3])
    o = (df.iloc[i, 4], df.iloc[i, 5])
    
    df.loc[i, "ED"] = np.round(haversine(e, d, unit = "km"), 4)
    df.loc[i, "DO"] = np.round(haversine(d, o, unit = "km"), 4)
    df.loc[i, "OE"] = np.round(haversine(o, e, unit = "km"), 4)

In [10]:
def get_outlier(df=None, column=None, weight=1.5):

    quantile_25 = np.percentile(df[column].values, 25)
    quantile_75 = np.percentile(df[column].values, 75)

    IQR = quantile_75 - quantile_25
    IQR_weight = IQR*weight
    lowest = quantile_25 - IQR_weight
    highest = quantile_75 + IQR_weight
    
    outlier_idx = df[column][ (df[column] < lowest) | (df[column] > highest) ].index
    
    return outlier_idx

In [11]:
iqr = []
for col in ["ED", "DO", "OE"]:
    iqr += list(get_outlier(df = df, column = col, weight = 1.5))
iqr = set(iqr)

df.drop(list(iqr), axis = 0, inplace = True)

In [12]:
df["Shape"] = 0
for i in range(len(df)):
    lst = sorted([df.iloc[i, -4], df.iloc[i, -3], df.iloc[i, -2]], reverse = True)
    a, b, c = lst[0]**2, lst[1]**2, lst[2]**2
    
    if a > b + c: df.iloc[i, -1] = "Obtuse"
    elif a == b + c: df.iloc[i, -1] = "Right"
    else: df.iloc[i, -1] = "Acute"

In [13]:
df["Area"] = 0
for i in range(len(df)):
    s = (df.iloc[i, -5] + df.iloc[i, -4] + df.iloc[i, -3]) / 2
    df.iloc[i, -1] = np.sqrt(s * (s - df.iloc[i, -5]) * (s - df.iloc[i, -4]) * (s - df.iloc[i, -3]))

In [14]:
df.dropna(inplace = True)
df.reset_index(inplace = True, drop = True)

In [15]:
df

,E_latitude,E_longitude,D_latitude,D_longitude,O_latitude,O_longitude,S_latitude,S_longitude,Station,Num,ED,DO,OE,Shape,Area
0,37.494519,127.108696,37.493823,127.120478,37.493571,127.120037,37.494408,127.121553,가락시장,3,1.0423,0.0479,1.0061,Obtuse,0.016059
1,37.493168,127.030014,37.498799,127.028976,37.500782,127.025754,37.497827,127.028614,강남,2,0.6329,0.3597,0.9264,Obtuse,0.078877
2,37.534123,127.092212,37.533969,127.093637,37.535500,127.093582,37.536994,127.084012,강변,2,0.1268,0.1704,0.1951,Acute,0.010659
3,37.498626,127.136567,37.493823,127.120478,37.497303,127.135637,37.496987,127.136077,개롱,5,1.5165,1.3922,0.1685,Obtuse,0.082586
4,37.493491,127.144168,37.493823,127.144994,37.492714,127.145523,37.493626,127.143682,거여,5,0.0817,0.1318,0.1475,Acute,0.005354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,37.548254,126.914825,37.548400,126.917303,37.550016,126.909618,37.549982,126.912175,합정,2,0.2191,0.7009,0.4991,Obtuse,0.025169
117,37.583577,127.000624,37.583235,126.998643,37.583965,127.002039,37.582496,127.002164,혜화,4,0.1787,0.3101,0.1319,Obtuse,0.001350
118,37.555592,126.920645,37.557758,126.925555,37.556062,126.923429,37.554709,126.920273,홍대입구,2,0.4953,0.2658,0.2509,Obtuse,0.018214
119,37.279044,126.998857,37.284080,126.984727,37.267764,127.001834,37.267466,127.002848,화서,1,1.3698,2.3628,1.2817,Obtuse,0.710221


In [17]:
df.to_csv("final_starbucks.csv", index = False)